# Sales Prediction
we will build a model to predict sales.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
import joblib # for save model

# Data Preprocessing

In [4]:
_data = pd.read_csv('./RoS_train.csv')
_data.head()

,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,month,year
0,5571,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,15
1,6714,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,15
2,5371,519,1,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,15
3,4257,449,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,15
4,4209,447,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,6,15


In [5]:
_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168607 entries, 0 to 168606
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Sales                      168607 non-null  int64  
 1   Customers                  168607 non-null  int64  
 2   Open                       168607 non-null  int64  
 3   Promo                      168607 non-null  int64  
 4   StateHoliday               168607 non-null  object 
 5   SchoolHoliday              168607 non-null  int64  
 6   StoreType                  168607 non-null  object 
 7   Assortment                 168607 non-null  object 
 8   CompetitionDistance        168179 non-null  float64
 9   CompetitionOpenSinceMonth  114961 non-null  float64
 10  CompetitionOpenSinceYear   114961 non-null  float64
 11  Promo2                     168607 non-null  int64  
 12  Promo2SinceWeek            83808 non-null   float64
 13  Promo2SinceYear            83

Sales is the target variable we want to predict.
columns that need to be checked for missing values are: 
* StateHoliday
* CompetitionDistance
* CompetitionOpenSinceMonth
* CompetitionOpenSinceYear
* Promo2SinceWeek
* Promo2SinceYear
* PromoInterval

In [8]:
for col in _data.columns:
    print(col)
    print(f"unique values count: {_data[col].unique().size}")
    print(_data[col].unique())
    print()

Sales
unique values count: 17585
[ 5571  6714  5371 ... 18325 23694 24043]

Customers
unique values count: 3238
[ 560  612  519 ... 3944 2709 2874]

Open
unique values count: 1
[1]

Promo
unique values count: 2
[1 0]

StateHoliday
unique values count: 4
['0' 'a' 'b' 'c']

SchoolHoliday
unique values count: 2
[1 0]

StoreType
unique values count: 4
['c' 'a' 'd' 'b']

Assortment
unique values count: 3
['a' 'c' 'b']

CompetitionDistance
unique values count: 655
[1.270e+03 5.700e+02 1.413e+04 6.200e+02 2.991e+04 3.100e+02 2.400e+04
 7.520e+03 2.030e+03 3.160e+03 9.600e+02 1.070e+03 1.300e+03 4.110e+03
 3.270e+03 5.000e+01 1.384e+04 3.240e+03 2.340e+03 5.500e+02 1.040e+03
 4.060e+03 4.590e+03 4.300e+02 2.300e+03 6.000e+01 1.200e+03 2.170e+03
 4.000e+01 9.800e+03 2.910e+03 1.320e+03 2.240e+03 7.660e+03 5.400e+02
 4.230e+03 1.090e+03 2.600e+02 1.800e+02 1.180e+03 2.900e+02 4.880e+03
 9.710e+03 2.700e+02 1.060e+03 1.801e+04 6.260e+03 1.057e+04 4.500e+02
 3.036e+04 7.170e+03 7.200e+02 6.620e+03

In [17]:
print("Percentage of missing values in each column:\n", 40*'-')
print(((_data.isnull().sum() / _data.shape[0]) * 100).sort_values(ascending=False))

Percentage of missing values in each column:
 ----------------------------------------
Promo2SinceYear              50.293879
Promo2SinceWeek              50.293879
PromoInterval                50.293879
CompetitionOpenSinceYear     31.817184
CompetitionOpenSinceMonth    31.817184
CompetitionDistance           0.253845
Sales                         0.000000
Promo                         0.000000
Customers                     0.000000
Assortment                    0.000000
StoreType                     0.000000
SchoolHoliday                 0.000000
StateHoliday                  0.000000
Open                          0.000000
Promo2                        0.000000
month                         0.000000
year                          0.000000
dtype: float64


we will remove these columns:
* Promo2SinceYear
* Promo2SinceWeek
* PromoInterval
* CompetitionOpenSinceYear
* CompetitionOpenSinceMonth